If you do not have this extension installed please type 

pip install ipython-sql

In [1]:
import sqlite3
conn = sqlite3.connect('fetchAssignment.db')

df = pd.read_csv('brands_new.csv')
df.to_sql('brands', conn, if_exists='replace', index=False)

df = pd.read_csv('receipts_new.csv')
df.to_sql('receipts', conn, if_exists='replace', index=False)

df = pd.read_csv('users_new.csv')
df.to_sql('users', conn, if_exists='replace', index=False)

df = pd.read_csv('receipt_items_new.csv')
df.to_sql('receipt_items', conn, if_exists='replace', index=False)

cursor = conn.cursor()

%load_ext sql
%sql sqlite:///fetchAssignment.db

'Connected: @fetchAssignment.db'

In [8]:
%%sql

SELECT ri.BRAND_CODE as name, SUM(ri.TOTAL_FINAL_PRICE) as price
FROM  receipt_items ri, receipts r
WHERE r.ID = ri.REWARDS_RECEIPT_ID AND strftime('%m', r.PURCHASE_DATE) = '06' AND ri.BRAND_CODE is not NULL
GROUP BY ri.BRAND_CODE
ORDER BY price DESC
LIMIT 1

 * sqlite:///fetchAssignment.db
Done.


name,price
KIRKLAND SIGNATURE,2571.8299999999977


In [3]:
%%sql

SELECT u.ID as user_id, SUM(r.TOTAL_SPENT) as price
FROM receipts r, users u
WHERE u.ID= r.USER_ID AND strftime('%m', r.PURCHASE_DATE) = '08'
GROUP BY r.USER_ID 
ORDER BY price DESC
LIMIT 1

 * sqlite:///fetchAssignment.db
Done.


user_id,price
609ab37f7a2e8f2f95ae968f,157739.13999999998


In [4]:
%%sql

SELECT r.USER_ID as user_id, (ri.TOTAL_FINAL_PRICE/ri.QUANTITY_PURCHASED) as price, ri.BRAND_CODE
FROM receipt_items ri, receipts r
WHERE r.ID = ri.REWARDS_RECEIPT_ID AND ri.QUANTITY_PURCHASED is not NULL AND price >= (
    SELECT  MAX(ri.TOTAL_FINAL_PRICE/ri.QUANTITY_PURCHASED)
FROM receipt_items ri, receipts r
WHERE r.ID = ri.REWARDS_RECEIPT_ID AND ri.QUANTITY_PURCHASED is not NULL
)
ORDER BY price DESC


 * sqlite:///fetchAssignment.db
Done.


user_id,price,BRAND_CODE
617376b8a9619d488190e0b6,31005.99,STARBUCKS
617376b8a9619d488190e0b6,31005.99,STARBUCKS


In [5]:
%%sql

SELECT  b.NAME as item , (ri.TOTAL_FINAL_PRICE/ri.QUANTITY_PURCHASED) as price
FROM brands b, receipt_items ri, receipts r
WHERE b.BRAND_CODE = ri.BRAND_CODE AND r.ID = ri.REWARDS_RECEIPT_ID AND ri.QUANTITY_PURCHASED is not NULL
ORDER BY price DESC
LIMIT 1

 * sqlite:///fetchAssignment.db
Done.


item,price
Starbucks,31005.99


In [6]:
%%sql

SELECT strftime('%m', r.PURCHASE_DATE) as month, COUNT(u.ID) as number_of_users, COUNT(DISTINCT u.ID) as number_of_dist_users
FROM users u, receipts r
WHERE u.ID = r.USER_ID AND r.PURCHASE_DATE is not NULL
GROUP BY month

 * sqlite:///fetchAssignment.db
Done.


month,number_of_users,number_of_dist_users
01,3819,97
02,3668,89
03,4762,89
04,4738,90
05,5344,89
06,5215,89
07,5927,89
08,6102,91
09,6295,90
10,7014,94
